In [ ]:
import numpy as np
import random
from copy import deepcopy

# Number of trucks and destinations
num_trucks = 3
num_destinations = 6

# Coordinates for each destination
destinations = np.array([
    [20, 30],  # D0
    [50, 60],  # D1
    [70, 20],  # D2
    [40, 80],  # D3
    [60, 90],  # D4
    [80, 50],  # D5
])

# Time windows for each destination (in hours)
time_windows = {
    0: (9, 12),
    1: (10, 14),
    2: (8, 11),
    3: (12, 15),
    4: (11, 13),
    5: (9, 13)
}

# PSO Parameters
num_particles = 10
num_iterations = 50
w = 0.5
c1 = 1.5
c2 = 1.5

# Euclidean distance
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# Convert permutation into truck routes
def decode_route(permutation):
    routes = []
    chunk_size = len(permutation) // num_trucks
    for i in range(num_trucks):
        start = i * chunk_size
        end = (i + 1) * chunk_size
        routes.append(permutation[start:end])
    return routes

# Fitness = total distance + time window penalties
def fitness(permutation):
    routes = decode_route(permutation)
    total_distance = 0
    penalty = 0
    for truck_id, route in enumerate(routes):
        if len(route) == 0:
            continue
        truck_route = destinations[route]
        for i in range(len(route) - 1):
            total_distance += euclidean_distance(truck_route[i], truck_route[i + 1])
        total_distance += euclidean_distance(truck_route[-1], truck_route[0])  # return trip

        for dest_idx in route:
            delivery_time = 9 + total_distance / 20  # simulate delivery time (hours)
            start_window, end_window = time_windows[dest_idx]
            if delivery_time < start_window or delivery_time > end_window:
                penalty += abs(delivery_time - start_window)

    return total_distance + penalty

# Initialize particles (each is a permutation of all destinations)
def initialize_particles():
    particles = []
    velocities = []
    base = list(range(num_destinations))
    for _ in range(num_particles):
        p = base[:]
        random.shuffle(p)
        particles.append(p)
        velocities.append([0] * len(p))  # Velocity in permutation space (placeholder)
    return particles, velocities

# Custom swap-based "velocity" operator for permutations
def update_particle(particle, velocity):
    new_particle = particle[:]
    for swap in velocity:
        i, j = swap
        new_particle[i], new_particle[j] = new_particle[j], new_particle[i]
    return new_particle

def subtract_permutations(p1, p2):
    """Generate swaps to convert p1 into p2"""
    swaps = []
    p1 = p1[:]
    for i in range(len(p1)):
        if p1[i] != p2[i]:
            j = p1.index(p2[i])
            swaps.append((i, j))
            p1[i], p1[j] = p1[j], p1[i]
    return swaps

# PSO Main
def pso():
    particles, velocities = initialize_particles()
    pbest = deepcopy(particles)
    pbest_fitness = [fitness(p) for p in particles]
    gbest = particles[np.argmin(pbest_fitness)]
    gbest_fit = min(pbest_fitness)

    for iter in range(num_iterations):
        print(f"Iteration {iter+1}, Best Fitness so far: {gbest_fit:.2f}")
        for i in range(num_particles):
            # Update velocity (list of swaps)
            cognitive = subtract_permutations(particles[i], pbest[i])
            social = subtract_permutations(particles[i], gbest)

            # Select random subsets of swaps to simulate movement
            vel = random.sample(cognitive, min(len(cognitive), 2)) + \
                  random.sample(social, min(len(social), 2))
            velocities[i] = vel

            # Update particle
            particles[i] = update_particle(particles[i], velocities[i])

            # Update personal best
            fit = fitness(particles[i])
            if fit < pbest_fitness[i]:
                pbest[i] = particles[i]
                pbest_fitness[i] = fit

                # Update global best
                if fit < gbest_fit:
                    gbest = particles[i]
                    gbest_fit = fit

    return decode_route(gbest), gbest_fit

# Run the PSO
best_route, best_fitness = pso()
print("\n🚛 Best route configuration:")
for i, truck in enumerate(best_route):
    print(f"  Truck {i+1}: Destinations {truck}")
print(f"\n⭐ Final Fitness (Distance + Penalty): {best_fitness:.2f}")

Iteration 1, Best Fitness so far: 252.66
Iteration 2, Best Fitness so far: 227.15
Iteration 3, Best Fitness so far: 227.15
Iteration 4, Best Fitness so far: 227.15
Iteration 5, Best Fitness so far: 227.15
Iteration 6, Best Fitness so far: 227.15
Iteration 7, Best Fitness so far: 227.15
Iteration 8, Best Fitness so far: 227.15
Iteration 9, Best Fitness so far: 227.15
Iteration 10, Best Fitness so far: 227.15
Iteration 11, Best Fitness so far: 227.15
Iteration 12, Best Fitness so far: 227.15
Iteration 13, Best Fitness so far: 227.15
Iteration 14, Best Fitness so far: 227.15
Iteration 15, Best Fitness so far: 227.15
Iteration 16, Best Fitness so far: 227.15
Iteration 17, Best Fitness so far: 227.15
Iteration 18, Best Fitness so far: 227.15
Iteration 19, Best Fitness so far: 227.15
Iteration 20, Best Fitness so far: 227.15
Iteration 21, Best Fitness so far: 227.15
Iteration 22, Best Fitness so far: 227.15
Iteration 23, Best Fitness so far: 227.15
Iteration 24, Best Fitness so far: 227.15
I